# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

Marketing Campaigns:

There are two main approaches for enterprises to promote products and/or services: through mass campaigns,
targeting general indiscriminate public or directedmarketing, targeting a specific set of contacts.
Nowadays, in a global competitive world, positive responses to mass campaigns are typically very low, 
less than 1%, according to the same study. Alternatively, directed marketing focus on targets that assumable will be
keener to that specific product/service, making this kind of campaigns more attractive due to its efficiency.
Nevertheless, directed marketing has some drawbacks, for instance it may trigger a negative attitude
towards banks due to the intrusion of privacy

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
import pandas as pd

In [7]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [8]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



1) Fields with implicit missing values

These columns encode “missing/unknown” using strings or sentinel values and should be handled explicitly:
	• job, marital, education, default, housing, loan
		○ Contain 'unknown' → this is a missing value placeholder.
		○ Recommended: convert 'unknown' to NaN (or keep as a separate category if you want to model “unknown” as informative).
	• pdays
		○ 999 means “not previously contacted” → this is a sentinel for “not applicable / unknown.”
		○ Recommended: set pdays = NaN when pdays == 999, and add a boolean flag (e.g., never_contacted_before = (pdays == 999) before coercion) so you don’t lose signal.
	• poutcome
		○ Values: 'failure', 'success', 'nonexistent'.
		○ 'nonexistent' means no prior campaign (not strictly missing), but it should be consistent with previous == 0 and pdays == 999.
		○ Recommended: keep as categorical, but validate consistency (see §3).

2) Fields that should be categorical (nominal/ordered)

Convert these from strings to category (with ordering where appropriate):
	• Nominal categorical:
		○ job, marital, education, default, housing, loan, contact, poutcome, y
		○ Note on y: convert to binary target (1 = 'yes', 0 = 'no') for modeling, keeping original as a label if desired.
	• Ordered categorical:
		○ month: ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
		○ day_of_week: ['mon','tue','wed','thu','fri']
		○ (Optional) education: you may impose an order if analytically justified:\ illiterate < basic.4y < basic.6y < basic.9y < high.school < professional.course < university.degree\ Be cautious—this imposes an ordinal structure that may not be universally appropriate.

3) Fields that should be numeric (and likely integer vs float)
Make sure these are correctly typed:
	• Integer (count-like):
		○ age, duration, campaign, previous, pdays
		○ Notes:
			§ duration is numeric but must not be used for realistic predictive modeling (it “leaks” outcome; use only for benchmarking as per the note).
			§ After handling pdays == 999 → NaN, you’ll typically store pdays as float (because of the NaNs) or keep as integer with a pandas nullable integer dtype.
	• Float (economic indicators):
		○ emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed
		○ These are continuous and should be float.

4) Cross-field consistency & sanity checks
Add these validations to catch data issues:
	• Previous contact coherence
		○ If previous == 0 → expect poutcome == 'nonexistent' and pdays == 999.
		○ If previous > 0 → expect poutcome ∈ {'failure','success'} and pdays < 999.
	• Bounds & domain checks
		○ campaign >= 1
		○ previous >= 0
		○ duration >= 0 (0 occurs and is valid; just note its strong correlation with y='no')
		○ pdays == 999 or pdays >= 0
		○ month ∈ {'jan',...,'dec'}
		○ day_of_week ∈ {'mon','tue','wed','thu','fri'}
	• Unknowns
		○ Review the incidence of 'unknown' in job, marital, education, default, housing, loan. If high, consider:
			§ Keeping 'unknown' as its own informative category, or
			§ Imputing / treating as NaN depending on your modeling strategy.

5) Recommended target and leakage handling
	• y (target): Convert to binary (1 for 'yes', 0 for 'no'), keep original label column for interpretability if useful.
	• duration: Exclude from training features for any realistic predictive setting (include only for benchmarking).


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

Business Objective:

Goal was to increase efficiency of directed campaigns for long-term depositsubscriptions by reducing the number of contacts to do.
During the Data Understanding phase, we analyzed the data main characteristics. 
The output presented in the reports of previous campaigns was composed of two values: the result and the amount of money invested.  
For this research, only the nominal result was accounted for, thus the goal is to predict if a client will subscribe the deposit.
Not regarding which amount is retained, turning it a classification task

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [12]:
# Core libraries
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# 1) Columns in scope (bank client data only)
bank_num = ['age']
bank_cat = ['job', 'marital', 'education', 'default', 'housing', 'loan']

# 2) Basic coercions (protect against accidental string types)
def coerce_and_clean_bank_info(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Coerce numeric
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    # Ensure categoricals are strings (so encoders behave consistently)
    for col in bank_cat + ['y']:
        if col in df.columns:
            df[col] = df[col].astype('string')
    return df

# 3) Build preprocessing for numeric & categorical
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    # (No scaler here; age in raw units is fine for tree models.
    # If you plan to use linear/logistic models, consider adding StandardScaler.)
])

categorical_transformer = Pipeline(steps=[
    # If you want to treat 'unknown' as missing, uncomment the next line and the replace step below
    # ('unknown_to_nan', FunctionTransformer(lambda X: X.replace('unknown', np.nan)))
    ('imputer', SimpleImputer(strategy='most_frequent')),  # handles NaN if present
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, bank_num),
        ('cat', categorical_transformer, bank_cat),
    ],
    remainder='drop'
)

# 4) Prepare X (features) and y (target)
def prepare_features_and_target(df_raw: pd.DataFrame):
    df = coerce_and_clean_bank_info(df_raw)

    # Map target y to binary 1/0
    if 'y' not in df.columns:
        raise KeyError("Target column 'y' not found in DataFrame.")
    y = df['y'].map({'yes': 1, 'no': 0}).astype('Int64')

    # Features: only bank info
    X = df[bank_num + bank_cat]

    return X, y

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [13]:
# Scikit-learn
from sklearn.model_selection import train_test_split

# -----------------------------
# 1) Define columns in scope
# -----------------------------
BANK_NUM = ['age']
BANK_CAT = ['job', 'marital', 'education', 'default', 'housing', 'loan']
BANK_INFO_COLS = BANK_NUM + BANK_CAT
TARGET_COL = 'y'

# -----------------------------
# 2) Coercion & preparation
# -----------------------------
def coerce_and_prepare_bank_info(df: pd.DataFrame):
    """
    - Coerces dtypes for bank info features
    - Maps y to binary (1='yes', 0='no')
    - Returns X (features) and y (Series)
    """
    df = df.copy()

    # Ensure required columns exist
    missing = set(BANK_INFO_COLS + [TARGET_COL]) - set(df.columns)
    if missing:
        raise KeyError(f"Missing required columns: {sorted(missing)}")

    # Numeric coercion
    df['age'] = pd.to_numeric(df['age'], errors='coerce')

    # Keep categorical as string (robust for downstream encoders)
    for col in BANK_CAT + [TARGET_COL]:
        df[col] = df[col].astype('string')

    # Target mapping
    y = df[TARGET_COL].map({'yes': 1, 'no': 0})
    if y.isna().any():
        # If any unexpected labels exist, raise a helpful error
        bad_labels = df.loc[y.isna(), TARGET_COL].unique().tolist()
        raise ValueError(f"Target y contains unexpected labels: {bad_labels}. Expected 'yes'/'no'.")

    # Features only from bank info
    X = df[BANK_INFO_COLS].copy()

    return X, y.astype('int64')

# -----------------------------
# 3) Stratified train/test split
# -----------------------------
def split_train_test_bank_info(df: pd.DataFrame,
                               test_size: float = 0.2,
                               random_state: int = 42):
    """
    Returns: X_train, X_test, y_train, y_test
    """
    X, y = coerce_and_prepare_bank_info(df)

    # Stratify on y to preserve class distribution across splits
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    roc_auc_score, average_precision_score, log_loss, brier_score_loss
)

# Assume you already have y_train, y_test as 0/1 (1='yes')
def compute_baselines(y_true: pd.Series, k_frac: float = 0.1):
    """
    Compute baseline metrics for classification/ranking.
    k_frac: fraction of the test population you plan to contact (e.g., top 10%).
    """
    y_true = pd.Series(y_true).astype(int)
    n = len(y_true)
    p = y_true.mean()  # prevalence

    # --- Majority-class classifier: predict all 0 (no)
    y_pred_majority = np.zeros(n, dtype=int)
    majority_acc = accuracy_score(y_true, y_pred_majority)
    majority_bal_acc = balanced_accuracy_score(y_true, y_pred_majority)
    majority_f1_pos = f1_score(y_true, y_pred_majority, zero_division=0)

    # --- Random/constant scores for threshold-free metrics
    # Use constant probability p (best constant in expectation)
    y_score_const = np.full(n, fill_value=p, dtype=float)

    # ROC AUC baseline (for random, it’s 0.5; many libs will error on constant scores; guard it)
    roc_auc_base = 0.5

    # PR AUC baseline equals prevalence
    pr_auc_base = p

    # Log loss baseline at constant p
    log_loss_base = log_loss(y_true, y_score_const, labels=[0,1])

    # Brier score baseline at constant p
    brier_base = brier_score_loss(y_true, y_score_const)

    # --- Ranking baselines for top-k targeting
    k = max(1, int(np.round(k_frac * n)))
    # Random ranking baseline expects precision@k ~= p, lift@k ~= 1
    precision_at_k_base = p
    lift_at_k_base = 1.0
    # Expected recall@k under random = (k/n)
    recall_at_k_base = min(1.0, k / n)

    return {
        "prevalence (positive rate)": p,
        "majority_class_accuracy": majority_acc,
        "majority_class_balanced_accuracy": majority_bal_acc,
        "majority_class_F1_positive": majority_f1_pos,
        "roc_auc_baseline": roc_auc_base,
        "pr_auc_baseline": pr_auc_base,
        "log_loss_baseline": log_loss_base,
        "brier_score_baseline": brier_base,
        f"precision@{int(k_frac*100)}%_baseline": precision_at_k_base,
        f"recall@{int(k_frac*100)}%_baseline": recall_at_k_base,
        f"lift@{int(k_frac*100)}%_baseline": lift_at_k_base,
        "note": "Compute these on y_test (not y_train)."
    }


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [17]:
# Core libraries
import numpy as np
import pandas as pd

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score,
    average_precision_score, classification_report, confusion_matrix
)

# -----------------------------
# 0) Configuration
# -----------------------------
CONTACT_RATE = 0.10     # evaluate precision/recall/lift at top 10% by default
TEST_SIZE = 0.20
RANDOM_STATE = 42

# -----------------------------
# 1) Define scope: bank info features and target
# -----------------------------
BANK_NUM = ['age']
BANK_CAT = ['job', 'marital', 'education', 'default', 'housing', 'loan']
FEATURES = BANK_NUM + BANK_CAT
TARGET = 'y'

# -----------------------------
# 2) Helpers: prepare data
# -----------------------------
def prepare_bank_features_and_target(df: pd.DataFrame):
    df = df.copy()

    # Ensure required columns exist
    missing = set(FEATURES + [TARGET]) - set(df.columns)
    if missing:
        raise KeyError(f"Missing required columns: {sorted(missing)}")

    # Coerce numeric
    df['age'] = pd.to_numeric(df['age'], errors='coerce')

    # Categoricals to string (robust for OHE)
    for col in BANK_CAT + [TARGET]:
        df[col] = df[col].astype('string')

    # Target mapping
    y = df[TARGET].map({'yes': 1, 'no': 0})
    if y.isna().any():
        bad = df.loc[y.isna(), TARGET].unique().tolist()
        raise ValueError(f"Unexpected target labels: {bad}. Expected 'yes'/'no'.")

    # Features
    X = df[FEATURES].copy()
    return X, y.astype(int)

# -----------------------------
# 3) Preprocessing & model
# -----------------------------
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    # Treat 'unknown' as a valid category; impute missing with most frequent
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, BANK_NUM),
        ('cat', categorical_transformer, BANK_CAT),
    ],
    remainder='drop'
)

# Logistic Regression baseline
logreg = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',     # handle imbalance
    solver='lbfgs',              # reliable for OHE inputs
    n_jobs=None,                 # n_jobs not used by lbfgs; leave as default
    random_state=RANDOM_STATE
)

# Full pipeline: preprocessing + model
clf = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', logreg)
])


# -----------------------------
# 4) Train/test split
# -----------------------------
def stratified_split(df: pd.DataFrame, test_size=TEST_SIZE, random_state=RANDOM_STATE):
    X, y = prepare_bank_features_and_target(df)
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)



### Problem 9: Score the Model

What is the accuracy of your model?

In [20]:


X_train, X_test, y_train, y_test = stratified_split(df, test_size=0.2, random_state=42)
    
# Fit
clf.fit(X_train, y_train)


test_accuracy = clf.score(X_test, y_test)  # same as accuracy_score(y_test, clf.predict(X_test))

print(f"Test accuracy: {test_accuracy:.4f}")

Test accuracy: 0.5849


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [26]:

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score

import time

# -----------------------------
# Models (default settings)
# -----------------------------
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=RANDOM_STATE),
    'KNN': KNeighborsClassifier(),                   # n_neighbors=5 by default
    'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'SVM': SVC()                                     # default kernel='rbf'
}

# -----------------------------
# Benchmark function
# -----------------------------
def fit_and_score_models(df: pd.DataFrame):
    X, y = prepare_bank_features_and_target(df)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )

    rows = []
    for name, base_model in models.items():
        clf = Pipeline(steps=[
            ('prep', preprocessor),
            ('model', base_model)
        ])

        t0 = time.perf_counter()
        clf.fit(X_train, y_train)
        t1 = time.perf_counter()
        train_time = t1 - t0

        # Train accuracy
        y_pred_train = clf.predict(X_train)
        train_acc = accuracy_score(y_train, y_pred_train)

        # Test accuracy
        y_pred_test = clf.predict(X_test)
        test_acc = accuracy_score(y_test, y_pred_test)

        rows.append({
            'Model': name,
            'Train Time (s)': round(train_time, 4),
            'Train Accuracy': round(train_acc, 4),
            'Test Accuracy': round(test_acc, 4),
        })

    results_df = pd.DataFrame(rows).sort_values('Test Accuracy', ascending=False).reset_index(drop=True)
    return results_df

# -----------------------------
# 6) Run benchmark
# -----------------------------
results = fit_and_score_models(df)
print(results.to_string(index=False))

              Model  Train Time (s)  Train Accuracy  Test Accuracy
                SVM          4.5336          0.8873         0.8874
                KNN          0.0480          0.8914         0.8786
      Decision Tree          0.1001          0.9171         0.8648
Logistic Regression          0.1534          0.5913         0.5849


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [ ]:

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, average_precision_score, roc_auc_score, balanced_accuracy_score, f1_score, make_scorer
)

# -----------------------------
# Define models and search spaces
# -----------------------------
base_lr = LogisticRegression(max_iter=1000, class_weight='balanced', solver='lbfgs', random_state=RANDOM_STATE)
base_knn = KNeighborsClassifier()
base_dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
base_svm = SVC(probability=True, random_state=RANDOM_STATE)  # probability True for PR/precision@k

pipelines = {
    "Logistic Regression": Pipeline([('prep', preprocessor), ('model', base_lr)]),
    "KNN": Pipeline([('prep', preprocessor), ('model', base_knn)]),
    "Decision Tree": Pipeline([('prep', preprocessor), ('model', base_dt)]),
    "SVM": Pipeline([('prep', preprocessor), ('model', base_svm)]),
}

param_grids = {
    "Logistic Regression": {
        'model__C': [0.1, 0.5, 1.0, 2.0, 5.0],
        'model__penalty': ['l2'],
        # 'model__solver': ['lbfgs', 'liblinear']  # liblinear for smaller problems; keep lbfgs for OHE
    },
    "KNN": {
        'model__n_neighbors': [3, 5, 7, 11, 21],
        'model__weights': ['uniform', 'distance'],
        'model__metric': ['euclidean', 'manhattan']
    },
    "Decision Tree": {
        'model__max_depth': [None, 5, 10, 20, 30],
        'model__min_samples_split': [2, 10, 50],
        'model__min_samples_leaf': [1, 5, 10, 20],
        'model__ccp_alpha': [0.0, 0.001, 0.01],
        'model__class_weight': [None, 'balanced']
    },
    "SVM": {
        'model__kernel': ['rbf'],
        'model__C': [0.5, 1.0, 2.0, 5.0],
        'model__gamma': ['scale', 0.1, 0.01]
    }
}

# -----------------------------
# Run searches and evaluate
# -----------------------------
def tune_and_compare(df: pd.DataFrame, primary_scorer='average_precision'):
    """
    primary_scorer: 'average_precision' (PR AUC), 'roc_auc', or use precision_at_k_scorer
    """
    if primary_scorer == 'precision_at_k':
        scorer = precision_at_k_scorer
    else:
        scorer = primary_scorer  # string accepted by sklearn scorers

    X, y = prepare_bank_features_and_target(df)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    rows = []
    best_estimators = {}

    for name, pipe in pipelines.items():
        grid = param_grids[name]
        search = GridSearchCV(
            estimator=pipe,
            param_grid=grid,
            scoring=scorer,
            cv=cv,
            n_jobs=-1,
            refit=True,
            verbose=0
        )

        t0 = time.perf_counter()
        search.fit(X_train, y_train)
        t1 = time.perf_counter()

        best_estimators[name] = search.best_estimator_
        train_time = t1 - t0

        # Evaluate on held-out test
        best_model = search.best_estimator_

        # Predict probabilities if available
        if hasattr(best_model.named_steps['model'], 'predict_proba'):
            y_proba_test = best_model.predict_proba(X_test)[:, 1]
        elif hasattr(best_model.named_steps['model'], 'decision_function'):
            # decision_function may output uncalibrated scores; still OK for ranking
            y_proba_test = best_model.decision_function(X_test)
        else:
            y_proba_test = best_model.predict(X_test)

        y_pred_test = (y_proba_test >= 0.5).astype(int) if y_proba_test.ndim == 1 else best_model.predict(X_test)
        y_pred_train = best_model.predict(X_train)

        # Metrics
        test_acc = accuracy_score(y_test, y_pred_test)
        train_acc = accuracy_score(y_train, y_pred_train)
        roc = roc_auc_score(y_test, y_proba_test) if len(np.unique(y_test)) == 2 else np.nan
        pr_auc = average_precision_score(y_test, y_proba_test)
        bal_acc = balanced_accuracy_score(y_test, y_pred_test)
        f1_pos = f1_score(y_test, y_pred_test, zero_division=0)

        # Top-k metrics
        n_test = len(y_test)
        k = max(1, int(round(CONTACT_RATE * n_test)))
        order = np.argsort(-y_proba_test)
        topk = order[:k]
        precision_at_k = y_test.to_numpy()[topk].sum() / k
        prevalence = y_test.mean()
        lift_at_k = (precision_at_k / prevalence) if prevalence > 0 else np.nan

        rows.append({
            'Model': name,
            'Best Params': search.best_params_,
            'CV Best Score (primary)': search.best_score_,
            'Train Time (s)': round(train_time, 3),
            'Train Accuracy': round(train_acc, 4),
            'Test Accuracy': round(test_acc, 4),
            'ROC AUC': round(roc, 4),
            'PR AUC': round(pr_auc, 4),
            f'Precision@{int(CONTACT_RATE*100)}%': round(precision_at_k, 4),
            f'Lift@{int(CONTACT_RATE*100)}%': round(lift_at_k, 2),
            'Balanced Acc': round(bal_acc, 4),
            'F1 (pos)': round(f1_pos, 4),
        })

    results_df = pd.DataFrame(rows).sort_values(['PR AUC', f'Precision@{int(CONTACT_RATE*100)}%'], ascending=False)
    return results_df, best_estimators

# -----------------------------
# Example usage:
# -----------------------------
results_pr, best_models_pr = tune_and_compare(df, primary_scorer='average_precision')  # PR AUC
print("=== Model Comparison (PR AUC as primary) ===")
print(results_pr.to_string(index=False))
#
results_patk, best_models_patk = tune_and_compare(df, primary_scorer='precision_at_k')  # Precision@k
print("\n=== Model Comparison (Precision@k as primary) ===")
print(results_patk.to_string(index=False))

Exception ignored in: <function ResourceTracker.__del__ at 0x1068a5bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1078adbc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x105c1dbc0>
Traceback (most recent call last

##### Questions